In [1]:
import sys
sys.path.append('/opt/ml/code')

import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from my_modules.transform import get_transform
from my_modules.dataset import TrainDataset, EvalDataset
from my_modules.inference import create_label

import torch
from torch.utils.data import DataLoader

In [38]:
# df_train_path = f'df/df_labled_train.csv'
# df_valid_path = f'df/df_labled_valid.csv'
df_test_path = '/opt/ml/input/data/eval/info.csv'

# df_train = pd.read_csv(df_train_path)
# df_valid = pd.read_csv(df_valid_path)
df_test = pd.read_csv(df_test_path)

In [39]:
def sum_label3(df):
    
    def _label(row):
        return row['age'] + 3*row['gender'] + 6*row['mask']

    df['ans'] = df.apply(_label, axis=1)
    return df[['ImageID', 'ans']]

In [40]:
device = 'cuda'

In [41]:
df_submit = df_test.copy()

In [42]:
import pickle
with open('pkl/gender_pseudo_ensemble_hard.pkl', 'rb') as f:
    df_submit['gender'] = pickle.load(f)

with open('pkl/final5.pkl', 'rb') as f:
    df_submit['age'] = pickle.load(f) # df_submit['age'] = pickle.load(f)

with open('pkl/mask_pseudo_ensemble.pkl', 'rb') as f:
    df_submit['mask'] = pickle.load(f)

In [9]:
df_submit['age'].value_counts()

0    5355
2    3729
1    3516
Name: age, dtype: int64

In [12]:
with open('pkl/ensembled_age5.pkl', 'rb') as f:
    aaa = pickle.load(f) # df_submit['age'] = pickle.load(f)

In [20]:
import numpy as np
np.bincount(aaa)

NameError: name 'aaa' is not defined

In [11]:
import numpy as np
np.bincount(age_pre)

array([5350, 3450, 3800])

In [6]:
df_submit.describe()

,ans,gender,age,mask
count,12600.0,12600.000000,12600.000000,12600.000000
mean,0.0,0.386746,0.941190,0.431508
std,0.0,0.487024,0.889365,0.728960
min,0.0,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.000000,0.000000
50%,0.0,0.000000,1.000000,0.000000
75%,0.0,1.000000,2.000000,1.000000
max,0.0,1.000000,2.000000,2.000000


In [16]:
df_submit['mask'].value_counts()

0    8946
1    1851
2    1803
Name: mask, dtype: int64

In [21]:
df_submit.age.value_counts()

0    5349
2    3870
1    3381
Name: age, dtype: int64

In [ ]:
0    5334
2    3659
1    3607

In [43]:
df_submit

,ImageID,ans,gender,age,mask
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,0,2,2
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,0,2,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,0,2,2
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,0,2,2
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,0,0,2
...,...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,0,2,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,1,1,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,1,0,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,0,1,0


In [30]:
df_submit

,ImageID,ans,mask,gender
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,2,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,0,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,2,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,2,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,2,0
...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,0,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,0,1
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,1,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,0,0


In [44]:
## sumup labels

df_submit = sum_label3(df_submit)

In [45]:
df_submit.to_csv('submit_pseudos_ensemble_final.csv', index=False)

In [35]:
df_submit

,ImageID,ans,gender,age,mask
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0,0,2,2
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0,0,2,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0,0,2,2
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0,0,1,2
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0,0,0,2
...,...,...,...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,0,0,2,0
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0,1,1,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,0,1,0,1
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,0,0,1,0


In [14]:
df_best = pd.read_csv('output_best.csv')
df_curr =pd.read_csv('submit_please.csv')

In [34]:
mask_gender =  df_best['ans'] // 3

In [40]:
(mask_gender//2).to_numpy()

array([2, 0, 2, ..., 1, 0, 1])

In [41]:
mask_gender =  df_best['ans'] // 3
m = (mask_gender // 2).to_numpy()
g = (mask_gender % 2).to_numpy()

import pickle

with open('pkl/mask_pred_best.pkl', 'wb') as f:
    pickle.dump(m, f)

with open('pkl/gender_pred_best.pkl', 'wb') as f:
    pickle.dump(g, f)